# Constructor
Bloque diseñado para generar la sql en función a la pregunta del usuario y un  lista de tablas relevantes 

### Librerías necesarias

In [ ]:
#import lib
import sys
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

### Conexión a DB local

In [ ]:
# Base de datos dummy para hacer las pruebas.
from langchain.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite://///home/llmuser/DB/laloss2.db",sample_rows_in_table_info=0)

### Carga del modelo sqlcoder en LlamaCpp

In [ ]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Make sure the model path is correct for your system!
llm_sqlcoder2 = LlamaCpp(
    model_path="/home/llamademo/models/sqlcoder2.Q5_K_S.gguf",
    temperature=0.0,
    max_tokens=10000,
    n_ctx=1024,
    top_p=1,
    n_threads = 12,
    callback_manager=callback_manager2
    verbose=True,  # Verbose is required to pass to the callback manager
)

### Definicion de prompt y chain a utilizar

In [ ]:
# Definicion de prompt y chain a utilizar

_sql_prompt = """Generate a SQL query to answer the following question: {input}.
"""

PROMPT_SUFFIX = """### Database Schema
This query will run on a database whose schema is represented in this string:

{table_info}

"""

SQLite_PROMPT = PromptTemplate(
    input_variables=["input", "table_info"],
    template=_sql_prompt + PROMPT_SUFFIX,
)

llm_chain = LLMChain(
    prompt=SQLite_PROMPT,
    llm=llm_sqlcoder2
)

### La lista de tablas se debe obtener del filtro

In [ ]:
filterTables = ['USERS', 'PROFESSION','P_PERSON'] # Ejemplo
relevantTables = db.get_table_info(filterTables)

### Query SQL

In [ ]:
question = "what is the profesion of user Joaquin"
response = llm_chain.run({'input': question, 'table_info': relevantTables})

 ### Ejemplo:
 SELECT p.name, pr.name_profession FROM p_person p JOIN profession pr ON p.fk_p_profession_id::integer = pr.profession_id WHERE p.name ilike '%Joaquin%'